### Sedaro API "Hello World" Example

This notebook exercises the most basic example of:

- Querying the API
- Creating a new block
- Running a scenario
- Viewing results


#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been
compromised, you can revoke it in the user settings interface on the Sedaro website.

This notebook also requires specific configuration of the HOST and scenario branch ids by
including the following in `config.json`

```json
{
  "HOST": "<HOST>",
  "WILDFIRE": {
    "AGENT_TEMPLATE_BRANCH_ID": "<BRANCH ID>",
    "SCENARIO_BRANCH_ID": "<BRANCH ID>"
    }
}


In [ ]:
import json

import matplotlib.pyplot as plt
from sedaro import SedaroApiClient

In [ ]:
# Settings
with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
AGENT_TEMPLATE_BRANCH_ID = config['WILDFIRE']['AGENT_TEMPLATE_BRANCH_ID']    # ID of the vehicle template branch
SCENARIO_BRANCH_ID = config['WILDFIRE']['SCENARIO_BRANCH_ID']                # ID of the scenario branch
HOST = config['HOST']                                                        # Sedaro instance URL

In [ ]:
def get_by_name(block_class, name):
    '''Search for a block with a particular name.'''
    instances = block_class.get_all()
    results = [entry for entry in instances if entry.name == name]
    if len(results) > 0:
        return results[0]
    else:
        return None

#### Instantiate Client

Instantiate the `SedaroApiClient` with our `API_KEY` and `HOST`, and fetch the desired branch. In this case, we'll get an agent template branch.


In [ ]:
sedaro = SedaroApiClient(api_key=API_KEY, host=HOST)

agent_template_branch = sedaro.agent_template(AGENT_TEMPLATE_BRANCH_ID)

#### Query Simulation Blocks

The API allows for queries of elements in the simulation, commonly referred to as "blocks". Here we query every block
of the `SubSystem` type that groups spacecraft components for analytics. For each subsystem, we print its name and the
name of every `Component` under it. Any element of the simulation can be queried in this manner.


In [ ]:
for subsystem in agent_template_branch.Subsystem.get_all():
    print(subsystem.name)
    for component in subsystem.components:
        print("    -", component.name)

#### Create Simulation Elements

Here a new `AngularVelocitySensor` is instantiated using the `branch_client.<BLOCK_NAME>.create` method. See the `sedaro` package documentation [here](https://github.com/sedaro/sedaro-python/tree/main/sedaro) for a listing of available
blocks.


In [ ]:
gnc_subsystem = get_by_name(agent_template_branch.Subsystem, 'GNC')
sensor_name = 'Test Sensor'
sensor = get_by_name(agent_template_branch.AngularVelocitySensor, sensor_name)
if sensor is None:
    sensor = agent_template_branch.AngularVelocitySensor.create(
        name=sensor_name,
        oneSigmaPerAxisError={'rad/s': 0.000175},
        subsystem=gnc_subsystem.id
    )

#### Simulate & Plot Results

First get the desired scenario branch and access the simulation on it.


In [ ]:
scenario_branch = sedaro.scenario(SCENARIO_BRANCH_ID)

sim = scenario_branch.simulation

Now we will use the API to start the simulation and plot some of the results. The results of this run will also appear in the web interface as usual.


In [ ]:
# Start scenario
sim.start(wait=True)

# Wait for it to finish, then get results
results = sim.results_poll()

# Summarize results
results.summarize()

In [ ]:
if results.success:
    for agent_name in results.templated_agents:
        agent_results = results.agent(agent_name)
        if sensor.id in agent_results:
            # Select sensor results and show summary
            sensor_result = agent_results.block(sensor.id)
            sensor_result.summarize()

            # Plot Data
            sensor_result.measurement.plot(show=False, marker='.', markersize=2, linewidth=0)
            plt.ylabel('Angular Velocity (rpm)')
            plt.legend(['x', 'y', 'z'])
            plt.show()